#  Решения в плоском канале

Меня захватила идея перейти от криволинейной системы координат ортогональной, исследовать решения в плоском канале. Это должно упростить как вычисления, так и их интерпретацию. Последние несколько дней я потратил на то, чтобы перенести существующие решения из круглой трубы в плоский канал, и вот что из этого получилось. 

## Условия на границах плоского канала

Решения, найденные в трубе, можно непрерывным образом перевести в решения для плоского канала. При этом направление $x$ остается продольным направление, $r$ становится нормальным к твердой стенке направлением (на твердой стенке $r=1$), $\theta$ переходит в транстверсальное направление. Ось трубы $r=0$ раскрывается в верхнюю стенку канала, условие на оси трубы переходит в условие проскальзывания на верхней стенке канала. 

$$
r=0 \rightarrow v = \frac{\partial u}{\partial r} = \frac{\partial w}{\partial r} = 0
$$

Другие условия не меняются. Периодичность по углу переходит в периодичность поперек течения, но ограничения, что период кратен 2pi больше нет. В силу условия зеркальной симметрии боковые стенки можно рассматривать как твердые стенки с проскальзыванием. 

## Модификация программы

Программа требует минимальных изменений. Вся информация о кривизне системы координат скрыта в массивах yt,rt из коммон блока /dimr/. В ортогональной системе координат они состоят только из единиц. Если ввести параметр $c$, который непрерывным образом меняется значение в массивах yt,rt от их исходного значение при c=0, до единиц при c=1, то двигаясь по нему методом Ньютона можно перевести уже найденные решения в решения для плоского канала. Изменения касаются только файла com.for. 

Кроме этого нужно дополнить граничные условия при r=0, и вместе с $с$ менять также расход в трубе от 1/2 до 2/3. 

Не забыл ли я что-то поменять в программе, я с уверенностью сказать не могу, так как много времени на проверку не тратил. Давление вычисляется верно (max div ~ 10e-14). В rp я менял только граничные условия. 

Если действовать так, как написано выше, то физический смысл имеют только решения при c=0 и с=1, а все промежуточные нужны только чтобы плавно перейти от одного у другому. Можно сделать так, чтобы и промежуточные решения имели смысл тоже. Можно считать, что течение заключено между двумя цилиндрами, внутренний цилиндр имеет радиус R, внешний R+1. На внутреннем цилиндре условия проскальзывания. Меняя непрерывно R от 0 до бесконечности, форма расчетной области меняется от трубы до плоского канала. За $c$ можно обозначить отношение длины внутренней дуги к внешней. 

## Реализация

Я выбрал бегущею волну, найденную недавно, и попробовал продвинуться по $c$ от 0 до 1. К сожалению, у меня не получилось. Я фиксировал Re и cf, а менял $c$ и nsym, рассуждая так, что меня интересует решение, находящееся на том же расстоянии от стенки, поэтому cf менять не стоит, но стоит менять nsym, так как заранее неясно, каким при этом будет расстояние между полосами и какой должна быть ширина расчетной области. Получилось так, что примерно при c=0.5 решение совершило разворот, и дальше продвинуться не удалось. Потом я пробовал менять пару c и Re, продвижение было, но крайне медленное, так что я отказался от этой идеи. 

В общем, это не значит, что пройти этот путь нельзя, но пока я оставил попытки и попробовал найти решение на сепаратрисе.

## Решение на сепатарисе

+ c = 1, расчеты проводятся в плоском канале
+ Xmax = 5R, я ищу бегущую волну
+ Zmax = 1.2R - длина расчетной области в направлении z (раньше $\theta$) 
+ Re = 2200
+ параметры сетки те же, что и раньше (64х40х32), epsr=0.25

При этих параметрах я нашел сперва турбулентный режим, а затем и решение на сепаратрисе. На графиках ниже приведены средние величины. 

<img src="edgeTW_Re2200_mean.png">

Скорость волны cf=0.97U. Средняя скорость на верхней стенке (y=1-r=0) составляет 0.93U. На графике ниже приведен средний профиль скорости

<img src="edgeTW_Re2200_profile.png" align="left"> 

На пристенную турбулентность решение не похожу, пульсации происходят непосредственно около верхней стенки канала, средний профиль скорости мало отличается от ламинарного. 

Я решил приводить к каждому решению среднюю амплитуду пульсаций, чтобы иметь возможность в будущем сравнить разные решения по этому показателю. В данном случае она равна 0.00965, вычислена по формуле

$$
\left(\frac{1}{V}\int_V u'u' + v'v' + w'w' d\tau\right)^{1/2}
$$

## Формирование продольных вихрей

Можно и в этом случае посмотреть, какой механизм отвечает за формирование средней завихренности. На графике ниже приведен вклад в образование OX от нелинейного взаимодействия пульсаций. Среднее течение не производит практически ничего (int = 0.0), поэтому приводить его я не стал.

<img src="pp_ox_gen.png">

Картинка построена также, как и раньше. Все умножено на Ox. На первой картинке суммарный вклад от всех слагаемых. На каждой из следующих вклад от одной из пар слагаемых. Наибольший вклад снова дает пара cux+d1x - конвекция вдоль трубы и растяжение вдоль трубы. 

В этом случае вклад всех слагаемых положительный, может быть этим объясняется то, что именно это решение лежит на сепаратрисе, то есть оказывается наиболее "живучим". 

## Заключение

Можно ли, основываясь на таком решении на сепаратрисе, делать какие-то содержательные выводы? Хотелось бы получить другую бегущею волну, существующею около стенки. Я не оставил идею перевести новую бегущую волну из трубы в канал методом Ньютона, мне кажется, это не бесполезно. Можно попробовать найти и другие решение, воспроизвести какое-нибудь решение из опубликованных. 

Решение на сепаратрисе в трубе, наверное, тоже стремится к оси трубы, но там у него нет возможности достичь ее, и оно останавливается на некотором расстоянии от нее. В таком случае то решение нельзя считать пристенным, как и это. 

Так как механизм образования продольных вихрей во всех случаях одинаков, не он определяет расстояние полос от стенки, и соответственно скорость их перемещения. 